In [ ]:
!pip install scikit-fuzzy
!pip install EasyGA

In [39]:
import skfuzzy as fuzz
from skfuzzy import control as ctrl
import numpy as np
from sklearn.pipeline import Pipeline
import re
import EasyGA
import random
import pandas as pd
import os

In [174]:
def fuzzyModelConstruction(chromosome):
  ##  For food quality
  # controller
  temperature = ctrl.Antecedent(np.linspace(0, 10, 11), 'temperature')
  flavor = ctrl.Antecedent(np.linspace(0, 10, 11), 'flavor')
  portion_size = ctrl.Antecedent(np.linspace(0, 10, 11), 'portion_size')
  attentiveness = ctrl.Antecedent(np.linspace(0, 10, 11), 'attentiveness')
  friendliness = ctrl.Antecedent(np.linspace(0, 10, 11), 'friendliness')
  speed = ctrl.Antecedent(np.linspace(0, 10, 11), 'speed')
  food_quality_tip = ctrl.Antecedent(np.linspace(0, 10, 11), 'food_quality_tip')
  service_quality_tip = ctrl.Antecedent(np.linspace(0, 10, 11), 'service_quality_tip')

  food_quality = ctrl.Consequent(np.linspace(0, 10, 11), 'food_quality')
  service_quality = ctrl.Consequent(np.linspace(0, 10, 11), 'service_quality')
  tip = ctrl.Consequent(np.linspace(0, 30, 31), 'tip')

  gene = chromosome[0]
  temperature['poor'] = fuzz.trimf(temperature.universe, [0, 0, gene.value[0]])
  temperature['average'] = fuzz.trimf(temperature.universe, [0, gene.value[1], 10])
  temperature['good'] = fuzz.trimf(temperature.universe, [gene.value[2], 10, 10])
  flavor['poor'] = fuzz.trimf(flavor.universe, [0, 0, gene.value[0]])
  flavor['average'] = fuzz.trimf(flavor.universe, [0, gene.value[1], 10])
  flavor['good'] = fuzz.trimf(flavor.universe, [gene.value[1], 10, 10])
  portion_size['poor'] = fuzz.trimf(portion_size.universe, [0, 0, gene.value[0]])
  portion_size['average'] = fuzz.trimf(portion_size.universe, [0, gene.value[1], 10])
  portion_size['good'] = fuzz.trimf(portion_size.universe, [gene.value[2], 10, 10])
  attentiveness['poor'] = fuzz.trimf(attentiveness.universe, [0, 0, gene.value[0]])
  attentiveness['average'] = fuzz.trimf(attentiveness.universe, [0, gene.value[1], 10])
  attentiveness['good'] = fuzz.trimf(attentiveness.universe, [gene.value[2], 10, 10])
  friendliness['poor'] = fuzz.trimf(friendliness.universe, [0, 0, gene.value[0]])
  friendliness['average'] = fuzz.trimf(friendliness.universe, [0, gene.value[1], 10])
  friendliness['good'] = fuzz.trimf(friendliness.universe, [gene.value[2], 10, 10])
  speed['poor'] = fuzz.trimf(speed.universe, [0, 0, gene.value[0]])
  speed['average'] = fuzz.trimf(speed.universe, [0, gene.value[1], 10])
  speed['good'] = fuzz.trimf(speed.universe, [gene.value[2], 10, 10])
  food_quality_tip['poor'] = fuzz.trimf(food_quality_tip.universe, [0, 0, gene.value[0]])
  food_quality_tip['average'] = fuzz.trimf(food_quality_tip.universe, [0, gene.value[1], 10])
  food_quality_tip['good'] = fuzz.trimf(food_quality_tip.universe, [gene.value[2], 10, 10])
  service_quality_tip['poor'] = fuzz.trimf(service_quality_tip.universe, [0, 0, gene.value[0]])
  service_quality_tip['average'] = fuzz.trimf(service_quality_tip.universe, [0, gene.value[1], 10])
  service_quality_tip['good'] = fuzz.trimf(service_quality_tip.universe, [gene.value[2], 10, 10])

  food_quality['poor'] = fuzz.trimf(food_quality.universe, [0, 0, 5])
  food_quality['average'] = fuzz.trimf(food_quality.universe, [0, 5, 10])
  food_quality['good'] = fuzz.trimf(food_quality.universe, [5, 10, 10])
  service_quality['poor'] = fuzz.trimf(service_quality.universe, [0, 0, 5])
  service_quality['average'] = fuzz.trimf(service_quality.universe, [0, 5, 10])
  service_quality['good'] = fuzz.trimf(service_quality.universe, [5, 10, 10])
  tip['low'] = fuzz.trimf(tip.universe, [0, 0, 15])
  tip['medium'] = fuzz.trimf(tip.universe, [0, 15, 30])
  tip['high'] = fuzz.trimf(tip.universe, [15, 30, 30])

  # Fuzzy Rules
  rule1_food = ctrl.Rule(temperature['poor'] | flavor['poor'] | portion_size['poor'], food_quality['poor'])
  rule2_food = ctrl.Rule(temperature['average'] | flavor['average'] | portion_size['average'], food_quality['average'])
  rule3_food = ctrl.Rule(temperature['good'] & flavor['good'] & portion_size['good'], food_quality['good'])
  rule1_service = ctrl.Rule(attentiveness['poor'] | friendliness['poor'] | speed['poor'], service_quality['poor'])
  rule2_service = ctrl.Rule(attentiveness['average'] | friendliness['average'] | speed['average'], service_quality['average'])
  rule3_service = ctrl.Rule(attentiveness['good'] & friendliness['good'] & speed['good'], service_quality['good'])
  rule1_tip = ctrl.Rule(food_quality_tip['poor'] | service_quality_tip['poor'], tip['low'])
  rule2_tip = ctrl.Rule(service_quality_tip['average'] | food_quality_tip['average'], tip['medium'])
  rule3_tip = ctrl.Rule(service_quality_tip['good'] & food_quality_tip['good'], tip['high'])

  # Control System Creation and Simulation
  food_ctrl = ctrl.ControlSystem([rule1_food, rule2_food, rule3_food])
  service_ctrl = ctrl.ControlSystem([rule1_service, rule2_service, rule3_service])
  tipping_ctrl = ctrl.ControlSystem([rule1_tip, rule2_tip, rule3_tip])

  food_sim = ctrl.ControlSystemSimulation(food_ctrl)
  service_sim = ctrl.ControlSystemSimulation(service_ctrl)
  tipping_sim = ctrl.ControlSystemSimulation(tipping_ctrl)

  return food_sim, service_sim, tipping_sim

In [14]:
def executeFuzzyInf(food_sim, service_sim, tip_sim, inputs):
  food_sim.input['temperature'] = inputs['temperature']
  food_sim.input['flavor'] = inputs['flavor']
  food_sim.input['portion_size'] = inputs['portion_size']
  service_sim.input['attentiveness'] = inputs['attentiveness']
  service_sim.input['friendliness'] = inputs['friendliness']
  service_sim.input['speed'] = inputs['speed']
  food_sim.compute()
  service_sim.compute()

  food_quality_output = food_sim.output['food_quality']
  service_quality_output = service_sim.output['service_quality']

  tip_sim.input['food_quality_tip'] = food_quality_output
  tip_sim.input['service_quality_tip'] = service_quality_output
  tip_sim.compute()

  tip_output = tip_sim.output['tip']

  # print(f"\nThe predicted tip is {tip_output:.2f}%")
  return tip_output

In [76]:
def dataCleaning(inputs):
  for key, value in inputs.items():
      inputs[key] = 0 if inputs[key] < 0 else inputs[key]
      inputs[key] = 10 if inputs[key] > 10 else inputs[key]
  return inputs

In [162]:
def fitness(chromosome):
  food_sim, service_sim, tip_sim = fuzzyModelConstruction(chromosome)
  total_error = 0
  for index, row in data.iterrows():
    inputs = {
        'temperature': row['food temperature']*10,
        'flavor': row['food flavor']*10,
        'portion_size': row['portion size']*10,
        'attentiveness': row['attentiveness']*10,
        'friendliness': row['friendliness']*10,
        'speed': row['speed of service']*10
    }
    inputs = dataCleaning(inputs)
    actual_tip = row['tip']
    predicted_tip = executeFuzzyInf(food_sim, service_sim, tip_sim, inputs)
    error = abs(actual_tip - predicted_tip)
    total_error += error
  return total_error

In [175]:
def generate_chromosome():
  chromosome = [
      random.uniform(0, 10),
      random.uniform(0, 10),
      random.uniform(0, 10)
  ]
  return chromosome

In [199]:
def pipeline(popSize, generaSize, generChrom=generate_chromosome):
  ga = EasyGA.GA()
  ga.gene_impl = lambda: generChrom()
  ga.chromosome_length = 1
  ga.population_size = popSize
  ga.target_fitness_type = 'min'
  ga.generation_goal = generaSize
  ga.fitness_function_impl = fitness

  # try:
  #   ga.evolve()
  # except Exception as e:
  #   print(f"Error: {type(e).__name__} {e}")
  # finally:
  #   file_paths = ['database.db', 'database.db-journal']
  #   for i in range(len(file_paths)):
  #     if os.path.exists(file_paths[i]):
  #         os.remove(file_paths[i])
  ga.evolve()

  if (generChrom == generate_chromosome):
    ga.print_best_chromosome()
    ga.print_worst_chromosome()
  else:
    ga.print_population()

In [202]:
global data
train_data = pd.read_csv("tipper_train.csv")
data = train_data
pipeline(40, 3)

Best Chromosome 	: [[4.686818788673647, 8.968116531313866, 3.5204170943083546]]
Best Fitness    	: 862.8462039459184
Worst Chromosome 	: [[8.140617946653116, 0.7522726066335239, 7.091546392859012]]
Worst Fitness    	: 916.6979666663922


# TEST CASES

In [203]:
def test_chromosome():
  best_chromosome = [4.686818788673647, 8.968116531313866, 3.5204170943083546]
  return best_chromosome

In [205]:
from EasyGA.structure import chromosome
global test_data
test_data = pd.read_csv("tipper_test.csv")
data = test_data
pipeline(1, 1, test_chromosome)

Chromosome - 0 [[4.686818788673647, 8.968116531313866, 3.5204170943083546]] / Fitness = 173.37130593427156

